In [1]:
from pathlib import Path
import re
import pandas as pd

In [2]:
text = Path('openelm_3B_in_domain_names.txt').read_text()

In [3]:
responses = text.split("\n\n\n")[:-1]

In [4]:
responses = [r.split("\n") for r in responses]

In [5]:
responses = [[re.findall(r"\d+: A: (.+)$", response[0])[0], re.findall(r"\d+: A: (.+)$", response[1])[0]] for response in responses]

In [6]:
responses

[['The answer is lp.',
  'The last letter of "SHEILA" is "I". The last letter of "Puckett" is "t". Concatenating them is "Ipt". The answer is ipt.'],
 ['The answer is ys.',
  'The last letter of "Tammy" is "y". The last letter of "Zamora" is "a". Concatenating them is "yza". The answer is yza.'],
 ['The answer is l.',
  'The last letter of "Pearl" is "l". The last letter of "Dicker" is "o". Concatenating them is "pl". The answer is pl.'],
 ['The answer is l.',
  'The last letter of "EVA" is "a". The last letter of "Ayala" is "l". Concatenating them is "al". The answer is al.'],
 ['The answer is ln.',
  'The last letter of "Dani" is "el". The last letter of "elle" is "le". Concatenating them is "dell". The answer is dell.'],
 ['The answer is tb.',
  'The last letter of "TARA" is "A". The last letter of "BEASLEY" is "Y". Concatenating them is "AY". The answer is AY.'],
 ['The answer is n.',
  'The last letter of "Joanne" is "n". The last letter of "Holland" is "d". Concatenating them is 

In [8]:
df = pd.DataFrame(responses, columns=['no_cot_answer', 'cot_answer'])

In [9]:
df.head()

,no_cot_answer,cot_answer
0,The answer is lp.,"The last letter of ""SHEILA"" is ""I"". The last l..."
1,The answer is ys.,"The last letter of ""Tammy"" is ""y"". The last le..."
2,The answer is l.,"The last letter of ""Pearl"" is ""l"". The last le..."
3,The answer is l.,"The last letter of ""EVA"" is ""a"". The last lett..."
4,The answer is ln.,"The last letter of ""Dani"" is ""el"". The last le..."


In [10]:
df["no_cot_answer"] = df["no_cot_answer"].str.lower()
df["cot_answer"] = df["cot_answer"].str.lower()

In [11]:
df.to_csv('openelm_in_domain_outputs.csv', index=False)

In [12]:
df = pd.read_csv('openelm_in_domain_outputs.csv')

In [13]:
df

,no_cot_answer,cot_answer
0,the answer is lp.,"the last letter of ""sheila"" is ""i"". the last l..."
1,the answer is ys.,"the last letter of ""tammy"" is ""y"". the last le..."
2,the answer is l.,"the last letter of ""pearl"" is ""l"". the last le..."
3,the answer is l.,"the last letter of ""eva"" is ""a"". the last lett..."
4,the answer is ln.,"the last letter of ""dani"" is ""el"". the last le..."
...,...,...
495,the answer is rn.,"the last letter of ""marie"" is ""e"". the last le..."
496,the answer is l.,"the last letter of ""carla"" is ""a"". the last le..."
497,the answer is t.,"the last letter of ""steve"" is ""e"". the last le..."
498,the answer is l.,"the last letter of ""alberto"" is ""to"". the last..."


In [14]:
og_data = pd.read_csv('in_domain_names.csv')

In [15]:
og_data = og_data[:len(df)]

In [16]:
df = og_data.join(df)

In [17]:
df["answer"] = df["answer"].str.lower()

In [18]:
df.head()

,question,answer,babbage_no_cot,babbage_cot,no_cot_answer,cot_answer
0,Take the last letters of the words in “SHEILA ...,at,A: The answer is sh.,"A: The last letter of ""Sheila"" is ""l"". The las...",the answer is lp.,"the last letter of ""sheila"" is ""i"". the last l..."
1,Take the last letters of the words in “TAMMY Z...,ya,A: The answer is tzm.,"A: The last letter of ""TAMMY"" is ""a"". The last...",the answer is ys.,"the last letter of ""tammy"" is ""y"". the last le..."
2,Take the last letters of the words in “PEARL D...,ln,A: The answer is pld.,"A: The last letter of ""PEARL"" is ""l"". The last...",the answer is l.,"the last letter of ""pearl"" is ""l"". the last le..."
3,Take the last letters of the words in “EVA AYA...,aa,A: The answer is e.,"A: The last letter of ""EVA"" is ""a"". The last l...",the answer is l.,"the last letter of ""eva"" is ""a"". the last lett..."
4,Take the last letters of the words in “DANIELL...,ed,A: The answer is ds.,"A: The last letter of ""Danielle"" is ""d"". The l...",the answer is ln.,"the last letter of ""dani"" is ""el"". the last le..."


In [19]:
def compute_accuracy(df, no_cot_col, cot_col, answer_col):
    no_cot_answers = df[no_cot_col].to_list()
    cot_answers = df[cot_col].to_list()
    answers = df[answer_col].str.lower()
    answers = answers.to_list()
    answers = zip(no_cot_answers, cot_answers, answers)
    answers = list(answers)
    answers = [[str(a[0]), str(a[1]), str(a[2])] for a in answers]
    answers = [
        [
            re.findall(r"\b(" + answer[2] + r")\b", answer[0]),
            re.findall(r"\b(" + answer[2] + r")\b", answer[1]),
            answer[2],
        ]
        for answer in answers
    ]
    answers = [[len(answer[0]), len(answer[1]), answer[2]] for answer in answers]
    no_cot = 0
    cot = 0
    for a, b, c in answers:
        if a != 0:
            no_cot += 1
        if b != 0:
            cot += 1

    no_cot_percent = no_cot / len(answers) * 100
    cot_percent = cot / len(answers) * 100

    print(f"No COT accuracy: {no_cot_percent}%")
    print(f"COT accuracy: {cot_percent}%")

In [20]:
compute_accuracy(df, 'no_cot_answer', 'cot_answer', 'answer')

No COT accuracy: 2.4%
COT accuracy: 6.2%
